## ResNet 모델 
- Residual Net 카이밍 허 등 개발, 2015년 우승
- 기존 20개층의 한계에서 152개 층 까지 늘림
- **ResNet 팀은 망을 100 layer 이상으로 깊게 하면서, 깊이에 따른 학습 효과를 얻을 수 있는 방법을 고민하였으며, 그 방법으로 Residual Learning이라는 방법을 발표**

## ResNet의 배경
- 1. 깊은 망의 문제점(Vanishing/Exploding Gradient 문제) 
    - CNN에서 파라미터 update를 할 때, gradient값이 너무 큰 값이거나 작은 값으로 포화되어 더 이상 변화가 없어 학습의 효과가 없어지거나, 학습 속도가 아주 느려지는 문제
    - **문제를 피하기 위해 batch normalization, 파라미터의 초기값 설정 방법 개선 등의 기법들이 적용되지만 layer의 개수가 일정 수를 넘어가게 되면 한계가 발생**

- 2. ResNet 팀의 배경 실험 (망이 깊어졌을 경우의 학습결과)
    - 모델이 깊어지면 적어도 얕은 모델만큼은 나와야 되는것이 아닌가? -> optimization에 문제가 있다고 생각.
    - 짧은 시간의 실험의 효과를 위해 **ImageNet보다는 간단한 CIFAR-10학습 데이터를 20-layer와 56-layer에 대하여 비교 실험 진행**
    - 실험결과로 보아 **학습 오차와 테스트 오차 모두 56-layer의 결과가 20-layer보다 나쁘게 나오는 것을 확인**
    
    
![resnet1](img/resnet1.png)

<hr>


## ResNet의 핵심원리
- 목표를 H(x)를 얻는 것이 아닌 출력과 입력의 차(H(x) - x, 잔차)를 얻는 것으로 목표를 수정
- 따라서 F(x) = H(x) - x 라면, 결과적으로 출력 H(x) = F(x) + x 가 된다, 그림에서의 F(x)는 그 뺀 차를 의미함.
- 왼쪽 블록이 오른쪽 블록처럼 표현될 수 있으며, Residual Learning의 기본 블록

![resnet_fxhx](img/resnet_fxhx.PNG)

- (1) X -> W -> Activation function ==> f(x) 임
- (2) h(x) = f(x) + x 로 표현되어질 수 있고 f(x) = h(x) - x 이다.
- (3) f(x)를 최소화하게 되면(하나의 일련의 과정을 최소화 하게 되면 h(x) - x가 최소가 된다는 뜻)

![resnet_차이점](img/resnet_차이점.png)

- resnet 18 vs 34 비교를 해본결과 잔차를 최소가 되게 학습한 효과가 보임.

![resnet18_vs_34](img/resnet18_vs_34.png)

<hr>

## 잔차 모듈 (Residual Modeule)
- **잔차 모듈 (residual module) : 기존신경망과 ResNet의 잔차 모듈 차이 즉 잔차를 학습함**
- **잔차 모듈의 특징은 기대하는 출력과 유사한 입력이 들어오면 영벡터에 가까운 값을 학습 => 입력의 작은 변화에 민감 => 잔차 학습** 
- **다양한 경로를 통해 복합적인 특징 추출 하고 필요한 출력이 얻어지면 컨볼루션 층을 건너뛸 수 있음. 또한 다양한 조합의 특징 추출 가능**
- shortcut은 파라미터가 없이 바로 연결이 되는 구조여서 연산량 관점에서는 덧셈이 추가되는 것 외에는 차이가 없다. (F(x) = H(x) + x)
- **입력에서 바로 출력으로 연결되는 shortcut 연결이 생기게 됨**

## Resnet 효과 (identity shortcut)
- (1) 이전에는 H(x)를 얻기 위한 학습 / 이제는 H(x) - x를 얻기 위한 학습을 하게 됨.
- (2) 최적의 경우 F(x)는 0이며, 학습할 방향이 미리 결정이 되고, 이것이 pre-conditioning 구실을 하게 됨
- (3) F(x)가 거의 0이 되는 방향으로 학습을 하게 되면, 입력의 작은 움직임(fluctuation)을 쉽게 검출 할 수 있다.
- (4) 입력과 같은 x가 그대로 출력에 연결되기 때문에 파라미터의 수에 영향이 없으며, 덧셈이 늘어나는 것을 제외하면 shortcut 연결을 통한 연산량 증가는 없다. 
- (5) 몇 개의 layer를 건너 뛰면서 입력과 출력이 연결이 되기 때문에 forward나 backward path가 단순해지는 효과를 얻을 수 있다.

<hr>


## ResNet 구조
- 망 설계시 VGGNet의 설계 철학을 많이 이용
- 대부분의 convolutional layer는 3x3 kernel을 갖도록 하였음
- 복잡도(연산량)을 줄이기 위해 max-pooling(1곳 제외), hidden fc, dropout등은 사용하지 않았음.

![resnet](img/resnet.png)

![resnet_structure](img/resnet_structure.png)

<hr>

<hr>

## Resnet Bottleneck Architecture
- 학습에 걸리는 시간을 고려하여 50, 101, 152 layer에 대해서는 기본구조를 조금 변경
- Dimension Reduction & Expansion
    - 1x1로 차원을 줄여주고 (계산량감소) 
    - 3x3 통과 후 1x1을 통한 차원 다시 증가
    - 256 -> 64 -> 64 -> 256 -> 256
    
![Resnet_Bottleneck_Architecture](img/Resnet_Bottleneck_Architecture.PNG)


<hr>

## 요약
- ShortCut 구조
    - 궁극적으로 Hx른 목표로 학습을 진행 그러나 Hx에 한번에 접근하기 어렵고 이를 위해 연산layer들을 거친 다음 Fx에 x(기존까지 학습된 정보)를 더한다. 따라서 목표로 하는 Hx = Fx + x가 최소인 상황을 향해 값을 학습
    - 이는 F(x)+x 와 F(x)(=궁극적인 Hx)의 잔차를 학습하는 모듈
    
- Bottleneck 구조 (1x1 64 -> 3x3 64 -> 1x1 256)
    - 첫번째 1x1 을 사용하는 건 기존 Image size정보를 64로 정보압축
    - 1x1 을 통해 파라미터를 줄일 여건이 만들어짐 (차원수가 줄어들음)
    - 줄인량을 통해 같은 컴퓨팅 자원과 시간자원으로 더 깊은 네트워크 설계 가능
    - 두번째 3x3 을 통해 기존에 하려던 Conv 연산 진행 (차원이 줄었으므로 연산량 이득)
    - 세번째 1x1을 통해 다시 차원을 증가(딥하게 감)
    - 즉 1x1을 통해 파라미터와 연산량을 줄여주고 더 깊게 쌓을 수 있게 됨

## 참고문헌
- https://kangbk0120.github.io/articles/2018-01/identity-mapping-in-deep-resnet
- https://blog.naver.com/laonple/220770760226
- https://lsjsj92.tistory.com/407